In [ ]:
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def filter_df(df, num_sources=None, num_targets=None):
    """
    A function to filter the data frame by top n sources and targets
    If num_sources or num_targets args are not supplied, they will not be filtered
    """
    if num_targets:
        top_targets = df.sum().sort_values(ascending=False)
        df = df[top_targets[:num_targets].index]
    
    if num_sources:
        top_sources = df.sum(axis=1).sort_values(ascending=False)[:num_sources]
        df = df.loc[top_sources.index]
    
    return df


def create_sankey_df(df, min_val=0):
    """
    Create the human-readable form of the Sankey chart data from an input data frame
    Data can be filtered by a threshold minimum value
    | Source | Source Value | Target | Target Value |
    |    A   |      5       |   i    |      3       |
    |    A   |      5       |   j    |      2       |
    |    B   |      7       |   i    |      1       |
    |    B   |      7       |   k    |      4       |
    """
    
    sources = []
    source_vals = []
    targets = []
    target_vals = []
    for source_name in df.index:
        row = df.loc[source_name]
        sources += [source_name] * sum(row.values > min_val)
        source_vals += [row[row.values > min_val].sum()] * sum(row.values > min_val)
        targets += list(row[row > min_val].index)
        target_vals += list(row[row > min_val].values)
    
    sankey_df = pd.DataFrame({
        'source': sources,
        'target': targets,
        'value': target_vals
    })
    
    return sankey_df


def create_label_dict(node_df, start_idx=0):
    """
    Return a dictionary with labels as keys and indices as values.  Applied 
    to each section of the flow visualization (two nodes and an edge). The
    `node_df` represents each section.
    """
    labels = set(node_df.source).union(node_df.target)
    sorted_labels = sorted(list(labels))
    return {sorted_labels[i]: i + start_idx for i in range(len(sorted_labels))}


def create_final_list(node_df, node_label_dict):
    """
    Return a list of dictionaries and the labels for each section of the flow 
    diagram.
    """
    df = pd.DataFrame({
        "source": node_df.source.map(node_label_dict),
        "target": node_df.target.map(node_label_dict),
        "value" : node_df.value
    })
    
    labels = list(node_label_dict.keys())
    return df.to_dict("records"), labels

In [ ]:
investor = pd.read_excel(
    "../data/Equity investor SUP matrix v2.xlsx",
    engine="openpyxl",
    skiprows=3,
    usecols="B, G:GL",
)

investor = investor.rename(columns={investor.columns[0]: "Ultimate Investor"})

# drop last row because it is a table summary
investor = investor[:-1]
investor = investor.set_index('Ultimate Investor')

# Set the value of Kingdom of Saudi Arabia investment in Saudi Arabian Oil company to equal
# the second largest investment
investor.loc['KINGDOM OF SAUDI ARABIA', ('Saudi Arabian Oil Co')] = investor.max(axis=0).sort_values(ascending=False)[1]

In [ ]:
financer = pd.read_excel(
    "../data/Financing SUP matrix v2.xlsx",
    engine="openpyxl",
    skiprows=4,
    usecols="A:AV",
)

# drop last row because it is null
financer = financer[:-1]
financer = financer.set_index('Banks')

In [ ]:
producer = pd.read_excel(
    "../data/MFA matrix v2.xlsx",
    #sheet_name="Conversion",
    engine="openpyxl",
    skiprows=1,
    usecols="C:FY",
).dropna()

producer = producer.groupby('Producer').sum()

In [ ]:
waste = pd.read_excel(
    "../data/MFA matrix v2.xlsx",
    #sheet_name="Waste",
    engine="openpyxl",
    skiprows=1,
    usecols="B, D:FY",
).dropna()

waste = waste.groupby('Country').sum()

In [ ]:
destination = pd.read_excel(
    "../data/MFA matrix v2.xlsx",
    sheet_name="Waste",
    engine="openpyxl",
    skiprows=1,
    usecols="C:FY",
).dropna()

destination = destination.groupby('Producer').sum()

In [ ]:
investor_df = create_sankey_df(investor)
financer_df = create_sankey_df(financer)
producer_df = create_sankey_df(producer)
waste_df = create_sankey_df(waste)
destination_df = create_sankey_df(destination)

In [ ]:
with open('../data/region_map.json', 'r') as f:
    region_dict = json.load(f)
country_to_region = { country['name']: country['region'] for country in region_dict }

In [ ]:
# Assign regions to countries
region = [country_to_region[country] for country in destination_df['target']]
destination_df['region'] = region
destination_df.head()

# Top Producer Bar Plot

In [ ]:
plt.figure(figsize=(4,3), dpi=150, facecolor=(1,1,1))
country_sum = destination_df.groupby('target').sum().sort_values(by='value', ascending=False)[:11].reset_index()
country_sum['percentage'] = country_sum['value'] /  destination_df.groupby('target').sum()['value'].sum() * 100
sns.barplot(x='target', y='percentage', data=country_sum)
plt.xticks(rotation=45, ha='right')
plt.ylabel('Global Plastics %')
plt.xlabel('')
plt.title('Percentage of Global Plastics Destination')
plt.show()

# Number of Export Countries for Top Producers
Top producers send plastics everywhere

In [ ]:
top_producers = list(destination_df.groupby('source').sum().sort_values(by='value', ascending=False).iloc[:11].index)

In [ ]:
num_countries = []
for producer in top_producers:
    num_countries.append(len(destination_df[destination_df['source'] == producer]))

In [ ]:
plt.figure(figsize=(4,3), dpi=150, facecolor=(1,1,1))
sns.barplot(x=top_producers, y=num_countries, color='C0')
plt.xticks(rotation=45, ha='right')
plt.xlabel('')
plt.title('Number of Countries of Export for Top Producers')
plt.ylabel('Number of Countries')
plt.show()

# Histogram of Number of Export Countries
Most producers send plastics indiscriminately across the globe

In [ ]:
top_producers = list(destination_df.groupby('source').sum().sort_values(by='value', ascending=False).index)
num_countries = []
for producer in top_producers:
    num_countries.append(len(destination_df[destination_df['source'] == producer]))

In [ ]:
plt.figure(figsize=(4,3), dpi=150, facecolor=(1,1,1))
sns.histplot(num_countries, bins=20, stat='probability')
plt.xlim([60, 145])
plt.xlabel('Number of Countries')
plt.ylabel('Fraction of Producers')
plt.title('Number of Export Countries')
plt.show()

# Per-Capita Plastics

In [ ]:
with open('../data/country_population.json', 'r') as f:
    population = json.load(f)

pop_countries = np.array([element['country'] for element in population])
populations = np.array([element['population'] for element in population])

In [ ]:
# Assign population to countries
country_total = destination_df.groupby('target').sum()
country_total['population'] = [np.squeeze(populations[pop_countries == country]) for country in country_total.index]
country_total['per capita'] = country_total['value'] / country_total['population'] * 1000000
country_total = country_total.sort_values(by='per capita', ascending = False)

In [ ]:
plt.figure(figsize=(4,3), dpi=150, facecolor=(1,1,1))
sns.barplot(x=country_total.index[:13], y=country_total['per capita'][:13])
plt.xticks(rotation=45, ha='right')
plt.ylabel('Kilograms of Plastic per Person')
plt.xlabel('')
plt.title('Countries with Highest Yearly Per Capita Plastics Use')
plt.show()

# Distribution of Per-Capita Plastics Use by Country
Most countries don't use that much plastic per person

In [ ]:
plt.figure(figsize=(4,3), dpi=150, facecolor=(1,1,1))
sns.histplot(data=country_total, x='per capita', bins=30)
plt.ylabel('Number of Countries')
plt.xlabel('Yearly Per-Capita Plastics Use (kg)')
plt.title('Distribution of Yearly Per-Capita Plastics Use')
plt.show()

# Fast Facts

In [ ]:
# Top plastics destinations
top_n = 20
country_sum = destination_df.groupby('target').sum().sort_values(by='value', ascending=False)[:top_n].reset_index()
country_sum['percentage'] = country_sum['value'] /  destination_df.groupby('target').sum()['value'].sum()
country_sum.loc[top_n] = {
    'target': 'Other',
    'value': destination_df.groupby('target').sum()['value'].sum() - country_sum['value'].sum(),
    'percentage': (destination_df.groupby('target').sum()['value'].sum() - country_sum['value'].sum()) / destination_df.groupby('target').sum()['value'].sum()
                   }
print("Top Plastics Destinations")
country_sum

In [64]:
#Plastics Use per Person per Year
country_total = country_total.sort_values(by='per capita', ascending = False)
print("Plastics Use per Person per Year")
country_total[:50]

Plastics Use per Person per Year


,value,population,per capita
target,,,
Luxembourg,142.379534,607950,234.196
Belgium,2293.021386,11433256,200.557
Singapore,820.331363,5638676,145.483
Taiwan,282.501282,2381677,118.614
Netherlands,1539.409882,17231624,89.3363
Kuwait,328.697283,4137309,79.4471
Israel,657.531861,8882800,74.023
United States,23433.099685,326687501,71.7294
Australia,1777.887393,24982688,71.1648


In [ ]:
# Top producers
top_producer_df = destination_df.groupby('source').sum().sort_values(by='value', ascending=False)
top_producer_df['percentage'] = top_producer_df['value'] / top_producer_df['value'].sum()
top_producer_df['value'].sum()

In [ ]:
country_total

In [ ]:
country_total.sort_values(by='value', ascending = False)

In [ ]:
country_total.sort_values(by='value', ascending = False)[:30]

In [65]:
for index, element in enumerate(destination.sum().sort_values(ascending=False).keys()):
    print(index + 1, element)

1 China
2 United States
3 India
4 Japan
5 United Kingdom
6 Brazil
7 France
8 Russian Federation
9 Indonesia
10 South Korea
11 Mexico
12 Vietnam
13 Germany
14 Spain
15 Australia
16 Italy
17 Canada
18 Thailand
19 Turkey
20 Saudi Arabia
21 Iran
22 Philippines
23 Netherlands
24 Chile
25 Egypt
26 Poland
27 South Africa
28 Pakistan
29 Nigeria
30 Argentina
31 Belgium
32 Colombia
33 Malaysia
34 United Arab Emirates
35 Peru
36 Israel
37 Switzerland
38 Singapore
39 Ukraine
40 Morocco
41 Hong Kong
42 Taiwan
43 Algeria
44 Czech Republic
45 Sweden
46 Romania
47 Bangladesh
48 Austria
49 Myanmar
50 Iraq
51 Oman
52 Greece
53 Portugal
54 Kenya
55 Finland
56 Ghana
57 Hungary
58 Denmark
59 New Zealand
60 Ireland
61 Kuwait
62 Kazakhstan
63 Belarus
64 Ecuador
65 Uzbekistan
66 Norway
67 Guatemala
68 El Salvador
69 Costa Rica
70 Slovakia
71 Venezuela
72 Dominican Republic
73 Tunisia
74 Lebanon
75 Tanzania
76 Luxembourg
77 Serbia
78 Cote d'Ivoire
79 Croatia
80 Uruguay
81 Sri Lanka
82 Yemen
83 Bulgaria
84 Para